In [1]:
from FoodRatingDataSet import FoodRatingDataset

SEQ_LEN = 10

data_set = FoodRatingDataset(seq_len=SEQ_LEN)

NUM_ING = data_set.NUM_ING
OTHER_FEATURES = data_set.OTHER_FEATURES
NUM_RECIPES = data_set.NUM_RECIPES

In [2]:
from FoodRecommenderModelSequence import FoodRecommenderModelSequence
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])

x = FoodRecommenderModelSequence(recipe_count=NUM_RECIPES, seq_len=SEQ_LEN)((ids, ing, ofe))

mod = keras.Model((ids, ing, ofe), x)
mod.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 10, 8023)]   0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 10, 3)]      0           []                               
                                                                                                  
 food_recommender_model_sequenc  (9, 161880)         35552348    ['input_1[0][0]',                
 e (FoodRecommenderModelSequenc                                   'input_2[0][0]',            

In [1]:
%load_ext tensorboard

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
import time

data, label = data_set.data()

mod.compile(loss=mod.loss, optimizer=mod.optimizer, metrics=['accuracy'])

tensorboard = TensorBoard(log_dir=f"logs/recommend_{int(time.time())}")
mod.fit(data, label, epochs=10, validation_split=0.3, callbacks=[tensorboard])

In [3]:
%tensorboard --logdir logs/

Launching TensorBoard...

In [3]:
from FoodRecommenderModel import FoodRecommenderModel
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])
tar_id, tar_ing, tar_ofe = tf.keras.Input(shape=[1]), tf.keras.Input(shape=[1, NUM_ING]), tf.keras.Input(
    shape=[1, OTHER_FEATURES])

x = FoodRecommenderModel()(((ids, ing, ofe), (tar_id, tar_ing, tar_ofe)))

mod = keras.Model((ids, ing, ofe, tar_id, tar_ing, tar_ofe), x)
mod.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 10, 8023)]   0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 10, 3)]      0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 1)]          0           []                               
                                                                                            

In [4]:
from FoodRecommenderModelSelfAttention import FoodRecommenderModelSelfAttention
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])
tar_id, tar_ing, tar_ofe = tf.keras.Input(shape=[1]), tf.keras.Input(shape=[1, NUM_ING]), tf.keras.Input(
    shape=[1, OTHER_FEATURES])

x = FoodRecommenderModelSelfAttention()(((ids, ing, ofe), (tar_id, tar_ing, tar_ofe)))

mod = keras.Model((ids, ing, ofe, tar_id, tar_ing, tar_ofe), x)
mod.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 10, 8023)]   0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 10, 3)]      0           []                               
                                                                                                  
 input_13 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            